In [113]:
def learning( userid, dataset):
    import numpy as np
    import pandas as pd
    import gc

    import matplotlib.pyplot as plt
    import seaborn as sns
    %matplotlib inline

    from sklearn import model_selection
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import RobustScaler

    import lightgbm as lgb
    import xgboost as xgb
    from catboost import CatBoostRegressor
    from catboost import CatBoostClassifier

    from IPython.display import display # Allows the use of display() for DataFrames

    import warnings
    warnings.filterwarnings('ignore')
    try:
        data = pd.read_csv(dataset,sep=";", columns=['timestamp','wind_speed','active_power'])
    except:
        try:
            data = pd.read_excel(dataset, columns=['timestamp','wind_speed','active_power'])
        except:
            TypeError("Неправильный формат файла.")
    data=data.dropna()
    data=data.drop(['timestamp'], axis=1)
    X = data.drop('active_power', axis=1)
    y = data['active_power']
    main = 'active_power'
    n = 50
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    dev_X, val_X, dev_y, val_y = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)
    y_test.index=list(range(len(y_test.index)))
    def run_lgb(train_X, train_y, val_X, val_y, test_X):
        params = {
            "objective" : "regression",
            "metric" : "rmse",
            "num_leaves" : 40,
            "learning_rate" : 0.004,
            "bagging_fraction" : 0.6,
            "feature_fraction" : 0.6,
            "bagging_frequency" : 6,
            "bagging_seed" : 42,
            "verbosity" : -1,
            "seed": 42
        }

        lgtrain = lgb.Dataset(train_X, label=train_y)
        lgval = lgb.Dataset(val_X, label=val_y)
        evals_result = {}
        model = lgb.train(params, lgtrain, 5000, 
                          valid_sets=[lgtrain, lgval], 
                          )

        pred_test_y = np.array(model.predict(test_X, num_iteration=model.best_iteration))
        return pred_test_y, model, evals_result
    def run_xgb(train_X, train_y, val_X, val_y, test_X):
        params = {'objective': 'reg:linear', 
              'eval_metric': 'rmse',
              'eta': 0.001,
              'max_depth': 10, 
              'subsample': 0.6, 
              'colsample_bytree': 0.6,
              'alpha':0.001,
              'random_state': 42, 
              'silent': True}

        tr_data = xgb.DMatrix(train_X, train_y)
        va_data = xgb.DMatrix(val_X, val_y)

        watchlist = [(tr_data, 'train'), (va_data, 'valid')]

        model_xgb = xgb.train(params, tr_data, 2000, watchlist, maximize=False, early_stopping_rounds = 100, verbose_eval=100)

        dtest = xgb.DMatrix(test_X)
        xgb_pred_y = np.array(model_xgb.predict(dtest))

        return xgb_pred_y, model_xgb
    def run_cat(dev_X, dev_y, val_X, val_y, X_test):
        cb_model = CatBoostRegressor(iterations=500,
                                 learning_rate=0.05,
                                 depth=10,
                                 eval_metric='RMSE',
                                 random_seed = 42,
                                 bagging_temperature = 0.2,
                                 od_type='Iter',
                                 metric_period = 50,
                                 od_wait=20)
        cb_model.fit(dev_X, dev_y,
                 eval_set=(val_X, val_y),
                 use_best_model=True,
                 verbose=50)
        pred_test_cat = np.array(cb_model.predict(X_test))
        return pred_test_cat, cb_model
    pred_test, model, evals_result = run_lgb(dev_X, dev_y, val_X, val_y, X_test)
    pred_test_cat, model_cat = run_cat(dev_X, dev_y, val_X, val_y, X_test)
    pred_test_xgb, model_xgb = run_xgb(dev_X, dev_y, val_X, val_y, X_test)
    import pickle
    for i in ['model_xgb','model', 'model_cat']:
        with open(f'{userid}_{i}.pkl','wb') as f:
            exec(f"clf = {i}")
            pickle.dump(clf,f)


    from keras.models import Sequential
    from keras.layers import LSTM
    from keras.layers import Dense
    from keras.layers import SimpleRNN
    from keras.layers import Dropout
    from keras.layers import GRU, Bidirectional
    from keras.optimizers.legacy import SGD
    import math

    from sklearn import metrics
    from sklearn.metrics import mean_squared_error
    def convert(data, main, n=50):
        training_data_len = math.ceil(len(data) * .8)
        train_data = data[:training_data_len].iloc[:,:] 
        test_data = data[training_data_len:].iloc[:,:]
        dataset_train = train_data[main].values 
        # Reshaping 1D to 2D array
        dataset_train = np.reshape(dataset_train, (-1,1)) 
        dataset_train.shape
        from sklearn.preprocessing import MinMaxScaler
        scaler = MinMaxScaler(feature_range=(0,1))
        # scaling dataset
        scaled_train = scaler.fit_transform(dataset_train)
        # Selecting Open Price values
        dataset_test = test_data[main].values 
        # Reshaping 1D to 2D array
        dataset_test = np.reshape(dataset_test, (-1,1))  
        # Normalizing values between 0 and 1
        scaled_test = scaler.fit_transform(dataset_test)
        X_train = []
        y_train = []
        for i in range(n, len(scaled_train)):
            X_train.append(scaled_train[i-n:i, 0])
            y_train.append(scaled_train[i-11:i+1, 0])
        X_test = []
        y_test = []
        # TODO y_test and y_train for 12 points
        # TODO rewrite for loop, to not get out of the scope error
        for i in range(n, len(scaled_test)):
            X_test.append(scaled_test[i-n:i, 0])
            y_test.append(scaled_test[i-11:i+1, 0])
        X_train, y_train = np.array(X_train), np.array(y_train)

        #Reshaping
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
    #     y_train = np.reshape(y_train, (y_train.shape[0],y_train))
        X_test, y_test = np.array(X_test), np.array(y_test)

        #Reshaping
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))
    #     y_test = np.reshape(y_test, (y_test.shape[0],1))    
        return X_train, y_train, X_test, y_test,train_data, test_data, scaler
    def all_model(X_train,y_train):    
        regressor = Sequential()

        # adding RNN layers and dropout regularization
        regressor.add(SimpleRNN(units = 12, 
                                activation = "tanh",
                                return_sequences = True,
                                input_shape = (X_train.shape[1],1)))
        regressor.add(Dropout(0.2))

        regressor.add(SimpleRNN(units = 12, 
                                activation = "tanh",
                                return_sequences = True))

        regressor.add(SimpleRNN(units = 12,
                                activation = "tanh",
                                return_sequences = True))

        regressor.add( SimpleRNN(units = 12))

        # adding the output layer
        regressor.add(Dense(units = 12,activation='sigmoid'))

        # compiling RNN
        regressor.compile(optimizer = SGD(learning_rate=0.01,
                                          decay=1e-6, 
                                          momentum=0.9, 
                                          nesterov=True), 
                          loss = "mean_squared_error")

        regressor.summary()

        return regressor
    X_train, y_train, X_test, y_test, test_data, train_data, scaler = convert(data, main, n)
    regressor = all_model(X_train,y_train)   
    regressor.fit(X_train,y_train,epochs=20,batch_size=200)
    import pickle
    for i in ['regressor']:
        with open(f'{userid}_{i}.pkl','wb') as f:
            exec(f"clf = {i}")
            pickle.dump(clf,f)
    last = X_test[0]
    with open(f'{userid}_last.npy', 'wb') as f:
        np.save(f, last)

In [112]:
def prediction_with(userid, town):
    for i in ['model_xgb','model', 'model_cat']:
    with open(f'{userid}_{i}.pkl', "rb") as file:
        deserialized_data = pickle.load(file)
        exec(f"{i} = deserialized_data")
    import asyncio
    import aiohttp
    from bs4 import BeautifulSoup as BS
    from fake_useragent import UserAgent

    import asyncio
    import aiohttp
    from bs4 import BeautifulSoup as BS
    from fake_useragent import UserAgent
    from datetime import datetime as dt
    import json
    import os
    import random
    import re
    import time
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    def get_data_forecast(url):

            headers = {"User-Agent": UserAgent().random}
            req = requests.get(url, headers)
            soup = BeautifulSoup(req.text, "lxml")
            week = soup.find_all("div",class_="p-flex p-flex_content_justify margin_bottom_20")
            l_wind, l_days = [],[]
            for i in range(56):
                l_days.append(dt.now()+timedelta(hours=6)-timedelta(hours = dt.now().hour, minutes=dt.now().minute))
            for w in week:     
                days=w.find_all("div",class_="p-flex__column p-flex__column_percent-16")   
                for d in days:
                    feat = d.find_all("span", class_="link__text")
                    l_wind.append(int(feat[2].text[:1]))

            df = pd.DataFrame({'timestamp': l_days,
                               'wind_speed':l_wind
            })
            return df

    df = (get_data_forecast(f"https://pogoda.mail.ru/prognoz/{town.lower()}/14dney/"))
    X =data.drop(['timestamp'], axis=1)
    pred_test = np.array(model.predict(X))
    pred_test_cat = np.array(model_xgb.predict(X))
    pred_test_xgb = np.array(cb_model.predict(X))
    def total_predict(pred_test,pred_test_xgb,pred_test_cat):
        sub = pd.DataFrame()
        sub_lgb = pd.DataFrame()
        sub_lgb["energy"] = pred_test

        sub_xgb = pd.DataFrame()
        sub_xgb["energy"] = pred_test_xgb

        sub_cat = pd.DataFrame()
        sub_cat["energy"] = pred_test_cat

        sub["energy"] = (sub_lgb["energy"] * 0.5 + sub_xgb["energy"] * 0.3 + sub_cat["energy"] * 0.2)
        return sub["energy"]
    total = total_predict(pred_test,pred_test_xgb,pred_test_cat)
    def plotting(y_test, pred_test):
        fig, axs = plt.subplots()
        #Plot for LightGBM predictions
        axs.plot( y_test, label = "test_data", color = "b")
        axs.plot(pred_test, label = "prediction ", color = "g")
        plt.ylabel('kWt')
        axs.legend()
        plt.savefig(f"{userid}_plot.png")
    plotting(y_test, total)
    df['active_power'] = total
    df.to_csv(f"{userid}_data.csv")

    

In [114]:
def prediction_without(userid):
    with open(f'{userid}_last.npy', 'rb') as f:
        last = np.load(f)
    for i in ['regressor']:
        with open(f'{userid}_{i}.pkl', "rb") as file:
            deserialized_data = pickle.load(file)
            exec(f"{i} = deserialized_data")
    def predict_row(X_test):
        y_RNN = regressor.predict(X_test)
        # scaling back from 0-1 to original
        y_RNN_O = scaler.inverse_transform(y_RNN) 
        return y_RNN_O 
    def plotting_row(test_data):
        fig = plt.subplots()

        #Plot for RNN predictions
        plt.plot(np.arange(test_data.index[-1]+1,test_data.index[-1]+13) , y_RNN_O[-1].reshape((12,1)), label = "y_RNN", color = "red", alpha=0.75)
        plt.title("Prediction of time row")
        plt.xlabel("Days")
        plt.ylabel("Wt")
        plt.show()  
        plt.savefig(f"{userid}_plot.png")
    y_RNN_O  = predict_row(last)
    plotting_row(test_data)
    l_days = []
    for i in range(12):
        l_days.append(dt.now()+timedelta(hours=6)-timedelta(hours = dt.now().hour, minutes=dt.now().minute))
    df = pd.DataFrame({'timestamp': l_days,
                               'active_power':y_RNN_O[-1]
            })
    df.to_csv(f'{userid}_data.csv')